# 06 Rhythm Profiles

In [ ]:
import os
import sys
print(sys.version)

In [ ]:
import pickle as pkl

import numpy as np
import matplotlib.pyplot as plt

import torch

import librosa

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'computing on {device}')

In [ ]:
sys.path.append("COMMON_UTILS/")

In [ ]:
from utils import plot_audio, play, plot_beat_grid
from drum_processor import getDownbeats
from drum_decomposition import (
    getDecomposition, plotDecomposition, drumGridFromHs, isolateSources, splitActivationsAndAudio, quantiseGrids,
)

In [ ]:
with open('./drum_templates.pkl', 'rb') as f:
    templates = pkl.load(f)
    
kd_temp = templates['kd_temp']
sd_temp = templates['sd_temp']
hh_temp = templates['hh_temp']

W_0 = np.stack([kd_temp, sd_temp, hh_temp], axis=1)
W_0 = torch.from_numpy(W_0).to(device)

**Problem:** Rhythm transfer from one target *song* to the source does not necessarily capture the style of a whole genre.
It also has the issue that currently it is the same percussion pattern throughout, so the original song structure is lost.

**Solution:** Build a target rhythm *profile* that represents a characteristic percussive track, and adapt the original drum track by selecting the probability of a drum hit in the new genre, given it was there in the original.
These profiles can be hand crafted for now, or produced through a data driven survey of real music.

Steps:
1. Discretise drum activations to produce a probability vector
2. Select target probability vector
3. Weighted geometric mean of each component to produce resulting probabilities
4. User selects threshold probaility to use in selecting from the estimated probability vector

In [ ]:
SRC = "rock2"

y_src, sr = librosa.load(f"EXAMPLE_MUSIC/{SRC}.wav", sr=44100)
y_drums, _ = librosa.load(f"PROCESSED/{SRC}/DRUMS/drums.wav", sr=sr)

In [ ]:
play(y_src, sr)
play(y_drums, sr)

In [ ]:
downbeats = getDownbeats(f"PROCESSED/{SRC}/source.wav")

In [ ]:
W, H, V, phi, net = getDecomposition(
    y_drums, R=3, trainable_W=True, W=W_0, device=device,
)

In [ ]:
plotDecomposition(W, H, V, sr=sr,)

In [ ]:
ys = isolateSources(net, device=device, phi=phi)

In [ ]:
for y in ys:
    play(y, sr)

In [ ]:
Hs, y_bars = splitActivationsAndAudio(ys, H, downbeats, sr, )

In [ ]:
grids = drumGridFromHs(Hs, grid_size=48)
grids = quantiseGrids(grids)
p_a = np.mean(grids, axis=0)

In [ ]:
plot_beat_grid(p_a);

In [ ]:
plt.plot(p_a[0])

In [ ]:
np.convolve()

In [ ]:
p_a.take()

In [ ]:
max_p = [np.max(np.take(p_a[0], np.arange(-3, 3)+i, mode="wrap")) for i in range(len(p_a[0]))]

In [ ]:
plt.plot(max_p)